# WMT18

* To english
* Segment level data
* Relative ranking (better translation vs. worse translation)
* Kendall score

In [ ]:
# install dependencies

!pip -q install datasets
!pip -q install bert_score
!pip -q install git+https://github.com/google-research/bleurt.git
!pip -q install unbabel-comet
!pip -q install transformers
!pip -q install POT

     |████████████████████████████████| 365 kB 4.7 MB/s 
     |████████████████████████████████| 115 kB 68.9 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
     |████████████████████████████████| 141 kB 71.9 MB/s 
     |████████████████████████████████| 212 kB 62.5 MB/s 
     |████████████████████████████████| 596 kB 53.2 MB/s 
     |████████████████████████████████| 127 kB 49.9 MB/s 
     |████████████████████████████████| 60 kB 3.3 MB/s 
     |████████████████████████████████| 4.7 MB 10.6 MB/s 
     |████████████████████████████████| 6.6 MB 54.3 MB/s 
     |████████████████████████████████| 352 kB 4.7 MB/s 
     |████████████████████████████████| 1.3 MB 63.8 MB/s 
     |████████████████████████████████| 64 kB 2.0 MB/s 
     |████████████████████████████████| 101 kB 5.7 MB/s 
     |████████████████████████████████| 9.5 MB 57.3 MB/s 
     |████████████████████████████████| 116 kB 20.6 MB/s 
     |████████████████████████████████| 409 kB 77.5 MB/s 
     |███████████████

In [ ]:
!git clone https://github.com/drehero/geneval

Cloning into 'geneval'...
remote: Enumerating objects: 508, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 508 (delta 77), reused 128 (delta 50), pack-reused 350
Receiving objects: 100% (508/508), 44.38 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (221/221), done.
Checking out files: 100% (176/176), done.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pathlib

import datasets
import numpy as np
import pandas as pd

from geneval.geneval.data.wmt import WMT18
from geneval.geneval.utils import kendall_score

In [ ]:
from geneval.reproducion.configs import baryscore_config as config

In [ ]:
for attr, val in vars(config).items():
    if not attr.startswith("_"):
        print(attr, ": ", val)

metric_name :  baryscore
metric_path :  ./geneval/geneval/metrics/baryscore/baryscore.py
uses_reference :  True
uses_source :  False
score_name :  baryscore_W
load_args :  {}
compute_args :  {'model_type': 'bert-base-uncased', 'batch_size': 128, 'last_layers': 5, 'use_idfs': True, 'sinkhorn_ref': 0.01}


In [ ]:
out_path = pathlib.Path("/content/drive/MyDrive/results/wmt18")
lang_pairs = ["cs-en", "de-en", "et-en", "fi-en", "ru-en", "tr-en", "zh-en"]

In [ ]:
scorer = datasets.load_metric(config.metric_path, **config.load_args)

In [ ]:
for lang_pair in lang_pairs:
    # load data
    wmt = WMT18(lang_pair, root="/tmp")

    # prepare input
    sents = []
    if config.uses_reference:
        refs = []
    if config.uses_source:
        srcs = []
    for i, sent in enumerate(wmt.translations_better):
        if sent not in sents:
            sents += [sent]
            if config.uses_reference:
                refs += [wmt.references[i]]
            if config.uses_source:
                srcs += [wmt.sources[i]]
    for i, sent in enumerate(wmt.translations_worse):
        if sent not in sents:
            sents += [sent]
            if config.uses_reference:
                refs += [wmt.references[i]]
            if config.uses_source:
                srcs += [wmt.sources[i]]

    # comput scores
    args = config.compute_args.copy()
    if config.uses_reference:
        args["references"] = refs
    if config.uses_source:
        args["sources"] = srcs

    scores = scorer.compute(predictions=sents, **args)
    sent2score = dict(zip(
        sents,
        scores[config.score_name] if config.score_name is not None else scores
    ))

    # save scores
    df = pd.DataFrame({
        "translation_better": wmt.translations_better,
        "translations_worse": wmt.translations_worse,
        "reference": wmt.references,
        "source": wmt.sources,
        "score_better": [sent2score[sent] for sent in wmt.translations_better],
        "score_worse": [sent2score[sent] for sent in wmt.translations_worse]
    })
    if "model_type" in args.keys():
        fn = f"{lang_pair}-{args['model_type'].split('/')[-1]}.csv"
    elif "config_name" in config.load_args.keys():
        fn = f"{lang_pair}-{config.load_args['config_name'].split('/')[-1]}.csv"
    else:
        fn = f"{lang_pair}.csv"
    df.to_csv(out_path / config.metric_name / fn, index=False)

Evaluating batch:   0%|          | 0/128 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ot/bregman.py:484: RuntimeWarning: divide by zero encountered in true_divide
  v = b / KtransposeU
/usr/local/lib/python3.7/dist-packages/ot/bregman.py:484: RuntimeWarning: overflow encountered in true_divide
  v = b / KtransposeU
/usr/local/lib/python3.7/dist-packages/ot/bregman.py:492: UserWarning: Warning: numerical errors at iteration 0
  warnings.warn('Warning: numerical errors at iteration %d' % ii)
Evaluating batch:   5%|▌         | 7/128 [00:00<00:08, 14.54it/s]/usr/local/lib/python3.7/dist-packages/ot/bregman.py:485: RuntimeWarning: divide by zero encountered in true_divide
  u = 1. / nx.dot(Kp, v)
Evaluating batch:   7%|▋         | 9/128 [00:00<00:10, 11.10it/s]/usr/local/lib/python3.7/dist-packages/ot/bregman.py:485: RuntimeWarning: overflow encountered in true_divide
  u = 1. / nx.dot(Kp, v)
/usr/local/lib/python3.7/dist-packages/ot/bregman.py:492: UserWarning: Warning: numerical

In [ ]:
# load scores and compute kendalls tau
results = {}
for lang_pair in lang_pairs:
    if "model_type" in config.compute_args.keys():
        fn = f"{lang_pair}-{config.compute_args['model_type'].split('/')[-1]}.csv"
    elif "config_name" in config.load_args.keys():
        fn = f"{lang_pair}-{config.load_args['config_name'].split('/')[-1]}.csv"
    else:
        fn = f"{lang_pair}.csv"
    df = pd.read_csv(out_path / config.metric_name / fn)
    results[lang_pair] = kendall_score(
        df["score_better"].to_list(),
        df["score_worse"].to_list()
    )

In [ ]:
results

{'cs-en': -0.3573385518590998,
 'de-en': -0.5203505931038028,
 'et-en': -0.37959485904691387,
 'fi-en': -0.28578732106339466,
 'ru-en': -0.32391387927720106,
 'tr-en': -0.26146627565982405,
 'zh-en': -0.25059207962346736}